<img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/>

# 构建包含多个深度神经网络 (DNN) 的 DeepStream 应用 #
您可构建 DeepStream 工作流水线来执行涉及多个神经网络的复杂分析。其中一个常见用例是：将检测器用作定位物体的主要推理引擎，并将分类器用作辅助推理引擎。此做法十分实用，因为分类模型通常更适用于检测帧内的单个物体。

## 学习目标 ##
在此 Notebook 中，您将学习如何使用 Python 构建涉及多个神经网络的 DeepStream 工作流水线，并实现以下目标：
* 规划工作流水线的架构
* 使用规范文件配置深度学习推理
* 处理元数据

**目录**
<br>
本 Notebook 涵盖以下部分：
1. [设计工作流水线](#s1)
    * [练习 #1 - 预览输入视频](#e1)
2. [深度学习模型准备工作](#s2)
    * [解压 TrafficCamNet 和 VehicleTypeNet 模型](#s2.1)
3. [构建视频 AI 应用](#s3)
    * [创建和连接工作流水线元件](#s3.1)
    * [练习 #2 - 初始化 GStreamer](#e2)
    * [练习 #3 - 创建元件](#e3)
    * [练习 #4 - 修改 GIE 配置文件](#e4)
    * [练习 #5 - 连接元件](#e5)
    * [练习 #6 - 向 OSD 汇点添加探针](#e6)
    * [练习 #7 - 启动工作流水线](#e7)
    * [查看推理](#s3.2)

<a name='s1'></a>
## 设计工作流水线 ##
构建视频 AI 应用，首先需要根据用例设计项目。在这项活动中，我们将构建 DeepStream 工作流水线，以根据停车场摄像头输入画面准确检测汽车，并对车辆类型进行分类。我们将使用提供的预训练模型。先看一下原始输入视频，然后使用基于 `ffmpeg` 命令行的实用程序来了解其格式。

<a name='e1'></a>
### 练习 #1 - 预览输入视频 ###
修改以下单元以研究输入视频。

In [ ]:
# DO NOT CHANGE THIS CELL
import os

# Set the input video path to an environment variable
os.environ['TARGET_VIDEO_PATH']='data/sample_30.h264'
os.environ['TARGET_VIDEO_PATH_MP4']='sample_30.mp4'

target_video_path=os.environ['TARGET_VIDEO_PATH']
target_video_path_mp4=os.environ['TARGET_VIDEO_PATH_MP4']

In [ ]:
# DO NOT CHANGE THIS CELL
from IPython.display import Video

# Convert the H.264 encoded video file to MP4 container file - this will generate the sample_30.mp4 file
!ffmpeg -i $TARGET_VIDEO_PATH $TARGET_VIDEO_PATH_MP4 \
        -y \
        -loglevel quiet

# View the input video
Video(target_video_path_mp4, width=720)

In [ ]:
!ffprobe -i <<<<FIXME>>>>

请注意下方输出中视频帧的**维度**和**帧速率**。

点击 ... 以显示**答案**。

<a name='s2'></a>
## 深度学习模型准备工作 ##
我们将使用 NGC 中的两个专用模型 - [TrafficCamNet](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/trafficcamnet) 物体检测模型和 [VehicleTypeNet](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/vehicletypenet) 分类模型。

<a name='s2.1'></a>
### 解压 TrafficCamNet 和 VehicleTypeNet 模型 ###
运行以下单元以解压文件

In [ ]:
!tar -xvf ngc_assets/trafficcamnet_vpruned_v1.0.tar -C ngc_assets/
!tar -xvf ngc_assets/vehicletypenet_vpruned_v1.0.tar -C ngc_assets/

<a name='s3'></a>
## 构建视频 AI 应用 ##
这是应用的工作流架构。我们将使用物体检测网络来识别和定位各帧中的汽车，然后进行辅助推理，对车辆类型进行分类。
<p><img src="images/deepstream_multi_gie_pipeline.png" width='1080'></p>

<a name='s3.1'></a>
### 创建和连接工作流水线元件 ###

<a name='e2'></a>
### 练习 #2 - 初始化 GStreamer ###
修改以下单元以初始化 GStreamer 并实例化工作流水线。

In [ ]:
# DO NOT CHANGE THIS CELL
# Import necessary GStreamer libraries and DeepStream python bindings
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst, GLib
from common.bus_call import bus_call
import pyds

In [ ]:
# Initialize GStreamer
Gst.<<<<FIXME>>>>

# Create Pipeline
pipeline=<<<<FIXME>>>>
print('Created pipeline')

点击 ... 以显示**答案**。

<a name='e3'></a>
### 练习 #3 - 创建元件 ###
修改以下单元以创建必要的工作流元件并设置其属性。您还需根据需要修改配置文件。

In [ ]:
# Create Source element for reading from a file and set the location property
source = Gst.ElementFactory.make("filesrc", "file-source")
source.set_property('location', "data/sample_30.h264")

# Create H264 Parser with h264parse as the input file is an elementary h264 stream
h264parser = Gst.ElementFactory.make("h264parse", "h264-parser")

# Create Decoder with nvv4l2decoder for accelerating decoding on GPU
decoder = Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder")

# Create Streamux with nvstreammux to form batches for one or more sources and set properties
streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
streammux.set_property('width', 888) 
streammux.set_property('height', 696) 
streammux.set_property('batch-size', 1)

# Create Primary GStreamer Inference Element with nvinfer to run inference on the decoder's output after batching
pgie=Gst.ElementFactory.make("nvinfer", "primary-inference")

# Create Secondary Inference Element with nvinfer to run inference on the pgie's output
sgie=<<<<FIXME>>>>

# Create Convertor to convert from YUV to RGBA as required by nvdsosd
nvvidconv1=Gst.ElementFactory.make("nvvideoconvert", "convertor1")

# Create OSD with nvdsosd to draw on the converted RGBA buffer
nvosd=Gst.ElementFactory.make("nvdsosd", "onscreendisplay")

# Create Convertor to convert from RGBA to I420 as required by encoder
nvvidconv2=Gst.ElementFactory.make("nvvideoconvert", "convertor2")

# Create Capsfilter to enforce frame image format
capsfilter=Gst.ElementFactory.make("capsfilter", "capsfilter")
caps=Gst.Caps.from_string("video/x-raw, format=I420")
capsfilter.set_property("caps", caps)

# Create Encoder to encode I420 formatted frames using the MPEG4 codec
encoder=Gst.ElementFactory.make("avenc_mpeg4", "encoder")
encoder.set_property("bitrate", 2000000)

# Create Sink with fakesink as the end point of the pipeline
sink=Gst.ElementFactory.make('filesink', 'filesink')
sink.set_property('location', 'output_04_raw.mpeg4')
sink.set_property("sync", 1)
print('Created elements')

点击 ... 以显示**答案**。

In [ ]:
# DO NOT CHANGE THIS CELL
# Add elements to pipeline
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(sgie)
pipeline.add(nvvidconv1)
pipeline.add(nvosd)
pipeline.add(nvvidconv2)
pipeline.add(capsfilter)
pipeline.add(encoder)
pipeline.add(sink)
print('Added elements to pipeline')

<a name='e4'></a>
### 练习 #4 - 修改 GIE 配置文件 ###
我们已为您提供[主要 GIE 配置文件](./spec_files/pgie_config_trafficcamnet_04.txt)。请将已启动的辅助 GIE 配置文件修改为 [spec_files/sgie_config_vehicletypenet_04.txt](./spec_files/sgie_config_vehicletypenet_04.txt)

点击 ... 以显示**答案**。

In [ ]:
# DO NOT CHANGE THIS CELL
# Set the location of the config file
pgie.set_property('config-file-path', 'spec_files/pgie_config_trafficcamnet_03.txt')
# sgie.set_property('config-file-path', 'spec_files/sgie_config_vehicletypenet_04.txt')
sgie.set_property('config-file-path', 'spec_files/sgie_config_vehicletypenet_04.txt')

In [ ]:
%%writefile ngc_assets/vehicletypenet_vpruned_v1.0/labels.txt

coupe;largevehicle;sedan;suv;truck;van

<p><img src='images/tip.png' width=720></p>

对于分类器，应以分号分隔 `labels.txt`。

<a name='e5'></a>
### 练习 #5 - 连接元件 ###
修改以下单元以连接元件。

In [ ]:
# Link elements together
source.link(h264parser)
h264parser.link(decoder)

# Link decoder source pad to streammux sink pad
decoder_srcpad=decoder.get_static_pad("src")    
streammux_sinkpad=streammux.get_request_pad("sink_0")
decoder_srcpad.link(streammux_sinkpad)

# Link the rest of the elements in the pipeline
streammux.link(pgie)
pgie.<<<<FIXME>>>>
<<<<FIXME>>>>.link(nvvidconv1)
nvvidconv1.link(nvosd)
nvosd.link(nvvidconv2)
nvvidconv2.link(capsfilter)
capsfilter.link(encoder)
encoder.link(sink)
print('Linked elements in pipeline')

点击 ... 以显示**答案**。

<a name='e6'></a>
### 练习 #6 - 向 OSD 接收衬垫添加探针 ###
修改以下单元以添加探针。

另外，我们还需要遍历辅助推理所生成的元数据。在本示例中，辅助推理是分类器对主要推理中的 `car` 类执行推理。我们可以在使用 `NvDsClassifierMeta.cast()` 投射元数据后访问 `classifier_meta_list` 中生成的元数据。根据辅助推理的次数，`NvDsObjectMeta` 对象可能包含一个或多个 `NvDsClassifierMeta` 对象。我们还需要投射到 `NvDsLabelInfo` 类，以获取辅助推理的结果分类。

In [ ]:
# DO NOT CHANGE THIS CELL
# Define the Probe Function
def osd_sink_pad_buffer_probe(pad, info):
    gst_buffer = info.get_buffer()

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list

    # Iterate through each frame in the batch metadata until the end
    while l_frame is not None:
        try:
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break

        frame_num=frame_meta.frame_num
        num_obj = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        print("Frame Number={} Number of Objects={}".format(frame_num, num_obj))
        
        # Iterate through each object in the frame metadata until the end
        while l_obj is not None:
            try:
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
                
                # Define an analyze_meta function to manipulate metadata
                analyze_meta(obj_meta)
            except StopIteration:
                break
                
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

In [ ]:
# DO NOT CHANGE THIS CELL
PGIE_CLASS_ID_CAR=0

# Define helper function
def analyze_meta(obj_meta): 
    # Only car supports secondary inference
    if obj_meta.class_id == PGIE_CLASS_ID_CAR:     
        cls_meta=obj_meta.classifier_meta_list
        
        # Iterate through each class meta until the end
        while cls_meta is not None:
            cls=pyds.NvDsClassifierMeta.cast(cls_meta.data)
            # Get label info
            label_info=cls.label_info_list  
            
            # Iterate through each label info meta until the end
            while label_info is not None:
                # Cast data type of label from pyds.GList
                label_meta=pyds.glist_get_nvds_label_info(label_info.data)
                if cls.unique_component_id==2:
                    print('\t Type & Probability = {}% {}'.format(round(label_meta.result_prob*100), label_meta.result_label))
                try:
                    label_info=label_info.next
                except StopIteration:
                    break
            
            try:
                cls_meta=cls_meta.next
            except StopIteration:
                break
    return None

In [ ]:
# Add probe to nvdsosd plugin's sink
osdsinkpad = nvosd.get_static_pad("sink")
osdsinkpad.add_probe(<<<<FIXME>>>>, <<<<FIXME>>>>)
print('Attached probe')

点击 ... 以显示**答案**。

<a name='e7'></a>
### 练习 #7 - 启动工作流水线 ###
修改以下单元，以启动 DeepStream 工作流水线。

In [ ]:
# DO NOT CHANGE THIS CELL
# Create an event loop
loop=GLib.MainLoop()

# Feed GStreamer bus messages to loop
bus=pipeline.get_bus()
bus.add_signal_watch()
bus.connect ("message", bus_call, loop)
print('Added bus message handler')

In [ ]:
print("Starting pipeline \n")
pipeline.set_state(<<<<FIXME>>>>)
try: 
    loop.<<<<FIXME>>>>
except: 
    pass

pipeline.set_state(Gst.State.NULL)

点击 ... 以显示**答案**。

<a name='s3.3'></a>
### 查看推理 ###

In [ ]:
# DO NOT CHANGE THIS CELL
# Convert MPEG4 video file to MP4 container file
!ffmpeg -i /dli/task/output_04_raw.mpeg4 /dli/task/output_04.mp4 \
        -y \
        -loglevel quiet

# View the output video
Video("output_04.mp4", width=720)

<img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/>